# Fidelity calculation using QuTIP

In [1]:
import numpy as np
import math
from typing import Optional, Tuple, cast, List, Dict
from qutip import *
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.quantum_info.states.utils import partial_trace
from qiskit.result.result import Result
import pickle

## Variables to control

In [2]:
# Filenames with results for 3 etas and their optimized parameters to LOAD
filename = '20210416a_C2_A2_500_10000'  
#filename = '20210413a_C2_A3_500_10000'  # 2 RX + 2 RY gates
n_channels = 2

## Auxiliary functions

In [3]:
""" save and load results to and from a file """
def save_results_to_disk(obj, name ):
    with open('results/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_results_from_file(name ):
    with open('results/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [4]:
def _prepare_initial_state_entangled(state_probability: float) -> Tuple[complex, complex, complex, complex]:
    """ Prepare initial state: computing 'y' as the amplitudes  """
    return (0, np.sqrt(state_probability), np.sqrt(1 - state_probability), 0)


In [121]:
def _run_all_circuits_and_return_density_matrices(configuration: dict) -> Tuple[List[int], List[int]]:
    """ Create a pair of Quantum Circuits, in its transpiled form, from a given configuration """

    state_vectors = [cast(Result, execute(_create_one_circuit(configuration, eta),
                                         backend=GLOBAL_BACKEND).result()).get_statevector()
                    for idx, eta in enumerate(configuration['eta_pair'])]
# ---------------------
#    for idx, eta in enumerate(configuration['eta_pair']):
#        print(ket2dm(Qobj(state_vectors[idx])))
    print(configuration['eta_pair'])
    d_m = [[], []] 
    d_m[0] = partial_trace(np.outer(state_vectors[0],state_vectors[0]), [2])  
    d_m[1] = partial_trace(np.outer(state_vectors[1],state_vectors[1]), [2])  
    print(type(d_m[0]))
    return d_m


In [122]:
#    print(configuration['eta_pair'])
#    print(state_vectors)
    d_m = [[], []] 
    for idx, eta in enumerate(configuration['eta_pair']):
        #print('idx = ' + str(idx))
        d_m[idx] = partial_trace(np.outer(state_vectors[idx],state_vectors[idx]), [2])
        print(QObj(d_m[idx]))
#        print(d_m)
    return d_m

IndentationError: unexpected indent (<ipython-input-122-ed481fcd1712>, line 3)

In [123]:
def _create_one_circuit(configuration: dict,
                        eta: float) -> QuantumCircuit:
    """ Creates one circuit from a given configuration and eta """
    qreg_q = QuantumRegister(3, 'q')
    creg_c = ClassicalRegister(2, 'c')

    initial_state = _prepare_initial_state_entangled(configuration['state_probability'])

    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.initialize(initial_state, [0, 1])
    circuit.reset(qreg_q[2])
    circuit.cry(2 * eta, qreg_q[1], qreg_q[2])
    circuit.cx(qreg_q[2], qreg_q[1])
    circuit.barrier()
    circuit.cx(qreg_q[0], qreg_q[1])
    circuit.h(qreg_q[0])
    circuit.rx(configuration['angle_rx1'], qreg_q[1])
    circuit.ry(configuration['angle_ry1'], qreg_q[1])
    circuit.rx(configuration['angle_rx0'], qreg_q[0])
    circuit.ry(configuration['angle_ry0'], qreg_q[0])
    circuit.measure([0, 1], creg_c)
    return circuit

## Fidelity calculations
At this point only pairs of etas are considered.


In [124]:
trio_results = load_results_from_file(name=filename)
GLOBAL_BACKEND=Aer.get_backend('statevector_simulator')
GLOBAL_ETA_TRIOS = trio_results['eta_pairs']
POVM_configurations = trio_results['configurations']
#density_matrices = [[], []]
fidelity_list = []
fidelity_results = trio_results
for i in range(len(GLOBAL_ETA_TRIOS)):
#    print(i)
    density_matrices = _run_all_circuits_and_return_density_matrices(POVM_configurations[i])
    #print(density_matrices[0])
    #print(density_matrices[1])
    print(Qobj(density_matrices[0]))
    print(Qobj(density_matrices[1]))
    
    #test = ket2dm(Qobj(density_matrices[0]))
    #print(test.ptrace(2))
    print(fidelity(Qobj(density_matrices[0]), Qobj(density_matrices[1])))


(0.07853981633974483, 0.0)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(0.15707963267948966, 0.0)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(0.23561944901923448, 0.0)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(0.3141592653589793, 0.0)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), typ

(1.335176877775662, 0.07853981633974483)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(1.413716694115407, 0.07853981633974483)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(1.4922565104551517, 0.07853981633974483)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(1.5707963267948966, 0.07853981633974483)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[

(0.39269908169872414, 0.3141592653589793)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(0.47123889803846897, 0.3141592653589793)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(0.5497787143782138, 0.3141592653589793)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(0.6283185307179586, 0.3141592653589793)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[

(1.335176877775662, 0.47123889803846897)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(1.413716694115407, 0.47123889803846897)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(1.4922565104551517, 0.47123889803846897)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(1.5707963267948966, 0.47123889803846897)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[

(1.0210176124166828, 0.7068583470577035)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(1.0995574287564276, 0.7068583470577035)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(1.1780972450961724, 0.7068583470577035)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(1.2566370614359172, 0.7068583470577035)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.

(1.413716694115407, 0.9424777960769379)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(1.4922565104551517, 0.9424777960769379)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(1.5707963267948966, 0.9424777960769379)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]
0.0
(1.0995574287564276, 1.0210176124166828)
<class 'qiskit.quantum_info.states.densitymatrix.DensityMatrix'>
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]